In [3]:
import numpy as np

UP, RIGHT, DOWN, LEFT = range(4)
ACTIONS = [UP, RIGHT, DOWN, LEFT]
ACTION_VECS = {UP: (-1, 0), RIGHT: (0, 1), DOWN: (1, 0), LEFT: (0, -1)}

def build_grid_mdp(env,
                   slip=0.0,
                   step_cost=0.0,
                   reward_goal=1.0,
                   reward_enemy=-1.0):

    n_rows, n_cols = len(env), len(env[0])

    idx = {}
    coords = []
    start_state = None
    terminals = set()

    s = 0
    for r in range(n_rows):
        for c in range(n_cols):
            idx[(r,c)] = s
            coords.append((r,c))
            if env[r][c] == 'S':
                start_state = s
            if env[r][c] in ('G', 'E'):
                terminals.add(s)
            s += 1

    nS = n_rows * n_cols
    nA = 4
    P = [[[] for _ in range(nA)] for _ in range(nS)]

    def in_bounds(r, c):
        return 0 <= r < n_rows and 0 <= c < n_cols

    def move_from(r, c, a):
        dr, dc = ACTION_VECS[a]
        nr, nc = r + dr, c + dc
        if not in_bounds(nr, nc):
            nr, nc = r, c  # bounce
        s2 = idx[(nr, nc)]
        cell = env[nr][nc]
        if cell == 'G':
            return s2, reward_goal, True
        if cell == 'E':
            return s2, reward_enemy, True
        return s2, step_cost, False

    # build transition list
    for s in range(nS):
        r, c = coords[s]

        # Terminal states
        if s in terminals:
            for a in ACTIONS:
                P[s][a] = [(1.0, s, 0.0, True)]
            continue

        # Non-terminal: compute stochastic mixture around intended action
        for a in ACTIONS:
            if slip == 0.0:
                s2, rew, done = move_from(r, c, a)
                P[s][a] = [(1.0, s2, rew, done)]
            else:
                # (1 - slip) to intended, split slip to perpendiculars (Left/Right turns)
                if a in (UP, DOWN):
                    perps = [LEFT, RIGHT]
                else:
                    perps = [UP, DOWN]

                dist = [
                    (1.0 - slip, a),
                    (slip / 2.0, perps[0]),
                    (slip / 2.0, perps[1]),
                ]

                # accumulate possibly duplicated next states
                acc = {}
                for p, a_eff in dist:
                    s2, rew, done = move_from(r, c, a_eff)
                    key = (s2, rew, done)
                    acc[key] = acc.get(key, 0.0) + p

                P[s][a] = [(p, s2, rew, done) for (s2, rew, done), p in acc.items()]

    return P, idx, coords, start_state, terminals


In [23]:
env = [['S',0,0,'E',0],
       [0,0,0,0,0],
       [0,0,0,0,0],
       [0,'E',0,'G',0],
       [0,0,0,0,0]]

P, idx, coords, s_start, terminals = build_grid_mdp(env,
                                                    slip=0.1,
                                                    step_cost=0.0,
                                                    reward_goal=1.0,
                                                    reward_enemy=-1.0)


In [24]:
P

[[[(0.9500000000000001, 0, 0.0, False), (0.05, 1, 0.0, False)],
  [(0.9, 1, 0.0, False), (0.05, 0, 0.0, False), (0.05, 5, 0.0, False)],
  [(0.9, 5, 0.0, False), (0.05, 0, 0.0, False), (0.05, 1, 0.0, False)],
  [(0.9500000000000001, 0, 0.0, False), (0.05, 5, 0.0, False)]],
 [[(0.9, 1, 0.0, False), (0.05, 0, 0.0, False), (0.05, 2, 0.0, False)],
  [(0.9, 2, 0.0, False), (0.05, 1, 0.0, False), (0.05, 6, 0.0, False)],
  [(0.9, 6, 0.0, False), (0.05, 0, 0.0, False), (0.05, 2, 0.0, False)],
  [(0.9, 0, 0.0, False), (0.05, 1, 0.0, False), (0.05, 6, 0.0, False)]],
 [[(0.9, 2, 0.0, False), (0.05, 1, 0.0, False), (0.05, 3, -1.0, True)],
  [(0.9, 3, -1.0, True), (0.05, 2, 0.0, False), (0.05, 7, 0.0, False)],
  [(0.9, 7, 0.0, False), (0.05, 1, 0.0, False), (0.05, 3, -1.0, True)],
  [(0.9, 1, 0.0, False), (0.05, 2, 0.0, False), (0.05, 7, 0.0, False)]],
 [[(1.0, 3, 0.0, True)],
  [(1.0, 3, 0.0, True)],
  [(1.0, 3, 0.0, True)],
  [(1.0, 3, 0.0, True)]],
 [[(0.9500000000000001, 4, 0.0, False), (0.05, 3

In [25]:
coords

[(0, 0),
 (0, 1),
 (0, 2),
 (0, 3),
 (0, 4),
 (1, 0),
 (1, 1),
 (1, 2),
 (1, 3),
 (1, 4),
 (2, 0),
 (2, 1),
 (2, 2),
 (2, 3),
 (2, 4),
 (3, 0),
 (3, 1),
 (3, 2),
 (3, 3),
 (3, 4),
 (4, 0),
 (4, 1),
 (4, 2),
 (4, 3),
 (4, 4)]

In [26]:
s_start

0

In [27]:
def value_iteration(P, gamma=0.99, theta=1e-10):
    nS, nA = len(P), len(P[0])
    V = np.zeros(nS, dtype=np.float64)

    while True:
        Q = np.zeros((nS, nA), dtype=np.float64)
        for s in range(nS):
            for a in range(nA):
                for prob, ns, rew, done in P[s][a]:
                    Q[s, a] += prob * (rew + (0.0 if done else gamma * V[ns]))
        V_new = np.max(Q, axis=1)
        if np.max(np.abs(V_new - V)) < theta:
            V = V_new
            break
        V = V_new

    best_actions = np.argmax(Q, axis=1)
    pi = lambda s: best_actions[s]
    return V, pi


In [16]:
V, pi = value_iteration(P, gamma=0.99)

arrow = {UP:'↑', RIGHT:'→', DOWN:'↓', LEFT:'←'}
n_rows, n_cols = len(env), len(env[0])

print("Optimal policy:")
for r in range(n_rows):
    row = []
    for c in range(n_cols):
        s = idx[(r,c)]
        cell = env[r][c]
        if s in terminals:
            row.append(cell)  # 'G' or 'E'
        else:
            row.append(arrow[pi(s)] if cell != 'S' else 'S')
    print(' '.join(str(x) for x in row))

print("\nState values (rounded):")
for r in range(n_rows):
    vals = []
    for c in range(n_cols):
        s = idx[(r,c)]
        vals.append(f"{V[s]:6.3f}")
    print(' '.join(vals))


Optimal policy:
S → ↓ E ↓
→ → → ↓ ↓
→ → → ↓ ↓
↑ E → G ←
→ → ↑ ↑ ↑

State values (rounded):
 0.951  0.961  0.970  0.000  0.970
 0.961  0.970  0.980  0.990  0.980
 0.970  0.980  0.990  1.000  0.990
 0.961  0.000  1.000  0.000  1.000
 0.970  0.980  0.990  1.000  0.990


In [28]:
V, pi = value_iteration(P, gamma=0.99)

arrow = {UP:'↑', RIGHT:'→', DOWN:'↓', LEFT:'←'}
n_rows, n_cols = len(env), len(env[0])

print("Optimal policy:")
for r in range(n_rows):
    row = []
    for c in range(n_cols):
        s = idx[(r,c)]
        cell = env[r][c]
        if s in terminals:
            row.append(cell)  # 'G' or 'E'
        else:
            row.append(arrow[pi(s)] if cell != 'S' else 'S')
    print(' '.join(str(x) for x in row))

print("\nState values (rounded):")
for r in range(n_rows):
    vals = []
    for c in range(n_cols):
        s = idx[(r,c)]
        vals.append(f"{V[s]:6.3f}")
    print(' '.join(vals))

Optimal policy:
S ↓ ← E ↓
→ → ↓ ↓ ↓
↑ ↑ → ↓ ↓
↑ E → G ←
→ → → ↑ ↑

State values (rounded):
 0.942  0.952  0.943  0.000  0.863
 0.952  0.964  0.976  0.986  0.977
 0.942  0.954  0.987  0.998  0.987
 0.830  0.000  0.998  0.000  0.998
 0.861  0.873  0.987  0.998  0.987
